In [1]:
import openai 
import pandas as pd 
import numpy as np
import json
from datetime import datetime
from dfply import *
from plotnine import *

In [2]:
conversations = pd.read_csv("cornell-dialog-data/movie_conversations.tsv", encoding='utf-8-sig', sep="\t", header = None)
titles = pd.read_csv("cornell-dialog-data/movie_titles_metadata.tsv", encoding='utf-8-sig', sep="\t", header = None)
lines_raw = pd.read_csv("cornell-dialog-data/movie_lines.tsv", encoding='utf-8-sig', header = None)
lines_list = lines_raw[0].str.split('\t')
characters_raw = pd.read_csv("cornell-dialog-data/movie_characters_metadata.tsv", encoding='utf-8-sig', header = None)
characters_list = characters_raw[0].str.split('\t')

In [3]:
lines = []
for i in range(0, len(lines_list)):
    v = lines_list[i]

    if len(v) <= 5:
        lines.append(v)

characters = []
for i in range(0, len(characters_list)):
    v = characters_list[i]

    if len(v) <= 6:
        characters.append(v)

lines = pd.DataFrame(lines)
characters = pd.DataFrame(characters)

In [4]:
conversations.columns = conversations.columns.map(str)
lines.columns = lines.columns.map(str)
titles.columns = titles.columns.map(str)
characters.columns = characters.columns.map(str)

In [7]:
#cleaning
lines['line_number'] = [int(x[1:]) for x in lines['0']]
lines = lines.sort_values(by = ['2', 'line_number'])

lines['next_movie'] = lines['2'].shift(1)
lines['next_movie'] = lines['next_movie'].fillna(lines['2'])

lines['line_number_shifted'] = lines.groupby(['2'])['line_number'].shift(1)
lines['diff'] = lines['line_number'] - lines['line_number_shifted']
lines['diff'] = lines['diff'].fillna(1.0)

lines

,0,1,2,3,4,line_number,next_movie,line_number_shifted,diff
86,L49,u0,m0,BIANCA,Did you change your hair?,49,m0,NaN,1.0
85,L50,u3,m0,CHASTITY,No.,50,m0,49.0,1.0
84,L51,u0,m0,BIANCA,You might wanna think about it,51,m0,50.0,1.0
648,L59,u9,m0,PATRICK,I missed you.,59,m0,51.0,8.0
647,L60,u8,m0,MISS PERKY,It says here you exposed yourself to a group o...,60,m0,59.0,1.0
...,...,...,...,...,...,...,...,...,...
48408,L300698,u1463,m99,INDIANA,The last Sankara Stone.,300698,m99,300691.0,7.0
48407,L300699,u1468,m99,WILLIE,And they don't even know what it really is.,300699,m99,300698.0,1.0
48406,L300700,u1463,m99,INDIANA,Well you didn't get your prince and there goes...,300700,m99,300699.0,1.0
48405,L300701,u1468,m99,WILLIE,You didn't do so well yourself. Finding that s...,300701,m99,300700.0,1.0


In [10]:
#create conversation id
conv_id = 0

def increment(r):
    global conv_id 
    
    if r[8] != 1.0:
        print('here')
        conv_id += 1
        return(conv_id)
    else: 
        if r[6] != r[2]:
            conv_id += 1
            return(conv_id)
        else:
            return(conv_id)

lines['conv_id'] = lines.apply(increment, axis = 1)

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


In [11]:
lines

,0,1,2,3,4,line_number,next_movie,line_number_shifted,diff,conv_id
86,L49,u0,m0,BIANCA,Did you change your hair?,49,m0,NaN,1.0,0
85,L50,u3,m0,CHASTITY,No.,50,m0,49.0,1.0,0
84,L51,u0,m0,BIANCA,You might wanna think about it,51,m0,50.0,1.0,0
648,L59,u9,m0,PATRICK,I missed you.,59,m0,51.0,8.0,1
647,L60,u8,m0,MISS PERKY,It says here you exposed yourself to a group o...,60,m0,59.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...
48408,L300698,u1463,m99,INDIANA,The last Sankara Stone.,300698,m99,300691.0,7.0,42369
48407,L300699,u1468,m99,WILLIE,And they don't even know what it really is.,300699,m99,300698.0,1.0,42369
48406,L300700,u1463,m99,INDIANA,Well you didn't get your prince and there goes...,300700,m99,300699.0,1.0,42369
48405,L300701,u1468,m99,WILLIE,You didn't do so well yourself. Finding that s...,300701,m99,300700.0,1.0,42369
